In [1]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.preprocessing import LabelEncoder

In [2]:
images=[]
globalpath=r'C:\Users\jerar\OneDrive\Desktop\food_classify\indian_foods\dataset\Dataset\train'
os.chdir(globalpath)
print(os.getcwd())
labels=os.listdir()#list of all foodnames since food name is the directory name
print(labels)
class_len=len(labels)
print(class_len)

C:\Users\jerar\OneDrive\Desktop\food_classify\indian_foods\dataset\Dataset\train
['burger', 'butter_naan', 'chai', 'chapati', 'chole_bhature', 'dal_makhani', 'dhokla', 'fried_rice', 'idli', 'jalebi', 'kaathi_rolls', 'kadai_paneer', 'kulfi', 'masala_dosa', 'momos', 'paani_puri', 'pakode', 'pav_bhaji', 'pizza', 'samosa']
20


In [5]:
data={"image":[],
    "label":[]
}
for label in labels:
    os.chdir(globalpath)
    path=os.getcwd()+"\\"+label
    os.chdir(path)
    imagelist=os.listdir()
    labellist=[]
    for i in range(0,len(imagelist)):
        imagelist[i]=path+"\\"+imagelist[i]
        labellist.append(label)
    data["image"].extend(imagelist)
    data["label"].extend(labellist)


'''
data["label"]=list(encoded_labels)'''
#load data into a DataFrame object:
df = pd.DataFrame(data)
df = df.sample(frac=1).reset_index(drop=True) # shuffling dataframe
print(df.head())



                                               image          label
0  C:\Users\jerar\OneDrive\Desktop\food_classify\...    masala_dosa
1  C:\Users\jerar\OneDrive\Desktop\food_classify\...    masala_dosa
2  C:\Users\jerar\OneDrive\Desktop\food_classify\...  chole_bhature
3  C:\Users\jerar\OneDrive\Desktop\food_classify\...      pav_bhaji
4  C:\Users\jerar\OneDrive\Desktop\food_classify\...           idli


In [7]:
#encoding labels
encoder = LabelEncoder()
encoded_labels= encoder.fit_transform(df.iloc[:,1])
df.iloc[:,1]=encoded_labels
df['label']=df['label'].astype(str)
print(df.head())


                                               image label
0  C:\Users\jerar\OneDrive\Desktop\food_classify\...    13
1  C:\Users\jerar\OneDrive\Desktop\food_classify\...    13
2  C:\Users\jerar\OneDrive\Desktop\food_classify\...     4
3  C:\Users\jerar\OneDrive\Desktop\food_classify\...    17
4  C:\Users\jerar\OneDrive\Desktop\food_classify\...     8


In [157]:
### creating tensors from dictionary using tf.data.Dataset.from_tensor_slices(tensors)
### tensors should be 1D arrays
###

In [9]:
train_df,test_df=train_test_split(df,test_size = 0.10, shuffle = True, random_state = 1)

In [11]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,validation_split = 0.2,rotation_range=30.horizontal_flip=True,fill_mode='nearest') # rescale + augmentation
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255) # only rescale

In [13]:
train_image = train_gen.flow_from_dataframe(dataframe = train_df,
                                           x_col = 'image',
                                           y_col = 'label',
                                           target_size = (224,224),
                                           batch_size = 16,
                                           color_mode = 'rgb',
                                           class_mode = 'categorical',
                                           shuffle = True,
                                           seed = 42,
                                           subset = 'training')
val_image = train_gen.flow_from_dataframe(dataframe = train_df,
                                           x_col = 'image',
                                           y_col = 'label',
                                           target_size = (224,224),
                                           batch_size = 16,
                                           color_mode = 'rgb',
                                           class_mode = 'categorical',
                                           shuffle = True,
                                           seed = 42,
                                           subset = 'validation')
test_image = test_gen.flow_from_dataframe(dataframe = test_df,
                                           x_col = 'image',
                                           y_col = 'label',
                                           target_size = (224,224),
                                           batch_size = 16,
                                           color_mode = 'rgb',
                                           class_mode = 'categorical',
                                           shuffle = False)

Found 2877 validated image filenames belonging to 20 classes.
Found 719 validated image filenames belonging to 20 classes.
Found 400 validated image filenames belonging to 20 classes.


In [ ]:
inputs = tf.keras.Input(shape = (120,120,3))
x = tf.keras.layers.Conv2D(filters = 16, kernel_size =(3,3), activation = 'relu' )(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size =(3,3), activation = 'relu' )(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation = 'relu')(x) #hidden layer
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
outputs = tf.keras.layers.Dense(class_len, activation = 'softmax')(x) #single neuron with linear activation function
model = tf.keras.Model(inputs = inputs, outputs = outputs)
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
history  = model.fit(train_image,
                    validation_data = val_image,
                    epochs = 50,
                    callbacks = [tf.keras.callbacks.EarlyStopping(
                    monitor = 'val_loss',
                    patience = 5,
                    restore_best_weights = True)])

Epoch 1/50
180/180 [==============================] - 109s 603ms/step - loss: 2.9577 - accuracy: 0.0730 - val_loss: 2.8636 - val_accuracy: 0.0946
Epoch 2/50
180/180 [==============================] - 112s 621ms/step - loss: 2.7861 - accuracy: 0.1185 - val_loss: 2.7615 - val_accuracy: 0.1085
Epoch 3/50
180/180 [==============================] - 104s 576ms/step - loss: 2.7197 - accuracy: 0.1265 - val_loss: 2.7315 - val_accuracy: 0.1446
Epoch 4/50
180/180 [==============================] - 106s 587ms/step - loss: 2.6848 - accuracy: 0.1453 - val_loss: 2.7045 - val_accuracy: 0.1377
Epoch 5/50
180/180 [==============================] - 108s 596ms/step - loss: 2.6378 - accuracy: 0.1582 - val_loss: 2.7915 - val_accuracy: 0.1057
Epoch 6/50
180/180 [==============================] - 109s 609ms/step - loss: 2.5847 - accuracy: 0.1773 - val_loss: 2.5990 - val_accuracy: 0.1766
Epoch 7/50
180/180 [==============================] - 109s 605ms/step - loss: 2.5353 - accuracy: 0.1835 - val_loss: 2.5812 -

In [162]:
result = model.evaluate(test_image, verbose = 0)
print('Test Loss : {:.3f}'.format(result[0]))
print('Test Accuracy : {:.3f}%'.format(result[1]*100))

Test Loss : 3.831
Test Accuracy : 9.250%


In [163]:
import os
os.getcwd()
os.chdir(r"C:\Users\jerar\OneDrive\Desktop\food_classify")
model.save("model.keras")

In [178]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
